In [1]:
# https://keras.io/examples/generative/wgan_gp/  
# 
import os
os.environ["CUDA_VISIBLE_DEVICES"] = str(0)


import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import *

from tqdm import tqdm
import matplotlib.pyplot as plt
import glob
from sklearn.utils import shuffle

from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint

# example of loading the generator model and generating images
from numpy import asarray
from numpy.random import randn
from numpy.random import randint
from tensorflow.keras.models import load_model
from matplotlib import pyplot


import tensorflow as tf


from tensorflow.keras.datasets.fashion_mnist import load_data
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import BatchNormalization

from tensorflow.keras import initializers

from skimage import io
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import glob
from sklearn.utils import shuffle

In [2]:
foldername2class = {
	'0.0':  0,
	'0.05': 1,
	'0.1':  2,
	'0.15': 3,
	'0.2':  4,
	'0.25': 5,
	'0.3':  6,
	'0.35': 7,
	'0.4':  8,
	'0.45': 9,
	'0.5':  10,
	'0.55': 11,
	'0.6':  12,
	'0.65': 13,
	'0.7':  14,
	'0.75': 15,
	'0.8':  16,
	'0.85': 17,
	'0.9':  18,
	'0.95': 19,
	'1.0':  20,
}

In [3]:
def preprocess_images(images):
  images = (images - 127.5) / 127.5
  return images.astype('float32')

def generator_img(path_list: list):
    counter = 0
    max_counter = len(path_list)
    while True:
        single_path = path_list[counter]
        label_s = foldername2class[single_path.split('/')[-2]]
        image_s = preprocess_images(np.asarray(io.imread(single_path), dtype=np.float32))
        yield image_s, label_s
        # yield np.ones((336, 336, 3))
        counter += 1

        if counter == max_counter:
            counter = 0
            path_list = shuffle(path_list)

def train_gen():
    return generator_img(train_images_path)

In [4]:
IMG_SHAPE = (336, 336, 3)
BATCH_SIZE = 12
N_CLASSES = 21
# Size of the noise vector
noise_dim = 128

PATH_DATA = '../../expand_double_modes'
SAVE_RESULT = 'exp_result'

train_images_path = []

iterator = tqdm(glob.glob(PATH_DATA + "/*"))
for single_folder in iterator:
    img_folder = shuffle(glob.glob(single_folder + '/*'))
    for indx, single_img_path in enumerate(img_folder):
        train_images_path.append(single_img_path)
iterator.close()

train_images_path = shuffle(train_images_path)

100%|██████████| 21/21 [00:00<00:00, 162.79it/s]


In [5]:
dataset = (
    tf.data.Dataset.from_generator(
        train_gen, 
        output_signature=(
            tf.TensorSpec(shape=IMG_SHAPE, dtype=np.float32),
            tf.TensorSpec(shape=(), dtype=np.int32),
        )
    )
    .shuffle(BATCH_SIZE * 10).batch(BATCH_SIZE)
)


In [6]:
train_size = len(train_images_path)

print(f'train: {train_size}')

train: 37800


In [7]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples, n_classes=21):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    z_input = x_input.reshape(n_samples, latent_dim)
    # generate labels
    labels = randint(0, n_classes, n_samples)
    return [z_input, labels]

In [8]:
def init_weights():
    return initializers.RandomNormal(stddev=0.02)

def init_weights():
    return None

In [9]:
# define the standalone discriminator model
def define_discriminator(in_shape, n_classes=21):
    # label input
    in_label = Input(shape=(1,))
    # embedding for categorical input
    li = Embedding(n_classes, 128)(in_label)
    # scale up to image dimensions with linear activation
    n_nodes = in_shape[0] * in_shape[1] * in_shape[2]
    li = Dense(n_nodes)(li)
    # reshape to additional channel
    li = Reshape((in_shape[0], in_shape[1], in_shape[2]))(li)
    # image input
    in_image = Input(shape=in_shape)
    # concat label as a channel
    merge = Concatenate()([in_image, li])                         # 336
    # downsample
    fe = Conv2D(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init_weights())(merge) # 168
    #fe = BatchNormalization()(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    # downsample
    fe = Conv2D(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init_weights())(fe)    # 84  
    #fe = BatchNormalization()(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    # downsample
    fe = Conv2D(256, (3,3), strides=(2,2), padding='same', kernel_initializer=init_weights())(fe)    # 42 
    #fe = BatchNormalization()(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    # downsample
    fe = Conv2D(256, (3,3), strides=(2,2), padding='same', kernel_initializer=init_weights())(fe)    # 21 
    #fe = BatchNormalization()(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    # downsample
    fe = Conv2D(256, (3,3), strides=(2,2), padding='same', kernel_initializer=init_weights())(fe)    # 10 
    #fe = BatchNormalization()(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    # flatten feature maps
    fe = layers.Flatten()(fe)
    fe = layers.Dropout(0.3)(fe)
    out_layer = layers.Dense(1)(fe)
    # define model
    model = keras.models.Model([in_image, in_label], out_layer)
    # compile model
    #opt = Adam(lr=0.0002, beta_1=0.5) # Adam(lr=0.0002, beta_1=0.5)
    #model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

# define the standalone generator model
def define_generator(latent_dim, n_classes=21, h_low=21, w_low=21):
    # label input
    in_label = Input(shape=(1,))
    # embedding for categorical input
    li = Embedding(n_classes, 128)(in_label)
    # linear multiplication
    n_nodes = h_low * w_low
    li = Dense(n_nodes)(li)
    # reshape to additional channel
    li = Reshape((h_low, w_low, 1))(li)
    # image generator input
    in_lat = Input(shape=(latent_dim,))
    # foundation for h_low x w_low image
    n_nodes = 64 * h_low * w_low
    gen = Dense(n_nodes)(in_lat)
    gen = LeakyReLU(alpha=0.2)(gen)
    gen = Reshape((h_low, w_low, 64))(gen)
    # merge image gen and label input
    merge = Concatenate()([gen, li])
    # upsample to 42
    gen = Conv2DTranspose(256, (3,3), strides=(2,2), padding='same', kernel_initializer=init_weights())(merge)  
    #gen = BatchNormalization(trainable=False)(gen)
    gen = LeakyReLU(alpha=0.2)(gen)
    # upsample to 84
    gen = Conv2DTranspose(256, (3,3), strides=(2,2), padding='same', kernel_initializer=init_weights())(gen)
    #gen = BatchNormalization(trainable=False)(gen)
    gen = LeakyReLU(alpha=0.2)(gen)
    # upsample to 168
    gen = Conv2DTranspose(256, (3,3), strides=(2,2), padding='same', kernel_initializer=init_weights())(gen)
    #gen = BatchNormalization(trainable=False)(gen)
    gen = LeakyReLU(alpha=0.2)(gen)
    # upsample to 336
    gen = Conv2DTranspose(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init_weights())(gen)
    #gen = BatchNormalization(trainable=False)(gen)
    gen = LeakyReLU(alpha=0.2)(gen)
    # output
    out_layer = Conv2D(IMG_SHAPE[-1], (3,3), activation='tanh', padding='same', kernel_initializer=init_weights())(gen)
    # define model
    model = keras.models.Model([in_lat, in_label], out_layer)
    return model

In [10]:
def conv_block(
    x,
    filters,
    activation,
    kernel_size=(3, 3),
    strides=(1, 1),
    padding="same",
    use_bias=True,
    use_bn=False,
    use_dropout=False,
    drop_value=0.5,
):
    x = layers.Conv2D(
        filters, kernel_size, strides=strides, padding=padding, use_bias=use_bias
    )(x)
    if use_bn:
        x = layers.BatchNormalization()(x)
    x = activation(x)
    if use_dropout:
        x = layers.Dropout(drop_value)(x)
    return x


def get_discriminator_model():
    img_input = layers.Input(shape=IMG_SHAPE)
    # Zero pad the input to make the input images size to (32, 32, 1).
    x = layers.ZeroPadding2D((2, 2))(img_input)
    x = conv_block(
        x,
        64,
        kernel_size=(5, 5),
        strides=(2, 2),
        use_bn=False,
        use_bias=True,
        activation=layers.LeakyReLU(0.2),
        use_dropout=False,
        drop_value=0.3,
    )
    x = conv_block(
        x,
        128,
        kernel_size=(5, 5),
        strides=(2, 2),
        use_bn=False,
        activation=layers.LeakyReLU(0.2),
        use_bias=True,
        use_dropout=True,
        drop_value=0.3,
    )
    x = conv_block(
        x,
        256,
        kernel_size=(5, 5),
        strides=(2, 2),
        use_bn=False,
        activation=layers.LeakyReLU(0.2),
        use_bias=True,
        use_dropout=True,
        drop_value=0.3,
    )
    x = conv_block(
        x,
        512,
        kernel_size=(5, 5),
        strides=(2, 2),
        use_bn=False,
        activation=layers.LeakyReLU(0.2),
        use_bias=True,
        use_dropout=False,
        drop_value=0.3,
    )

    x = layers.Flatten()(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(1)(x)

    d_model = keras.models.Model(img_input, x, name="discriminator")
    return d_model


d_model = define_discriminator(IMG_SHAPE)
d_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 1, 128)       2688        input_1[0][0]                    
__________________________________________________________________________________________________
dense (Dense)                   (None, 1, 338688)    43690752    embedding[0][0]                  
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 336, 336, 3) 0                                            
______________________________________________________________________________________________

In [11]:
def upsample_block(
    x,
    filters,
    activation,
    kernel_size=(3, 3),
    strides=(1, 1),
    up_size=(2, 2),
    padding="same",
    use_bn=False,
    use_bias=True,
    use_dropout=False,
    drop_value=0.3,
):
    x = layers.UpSampling2D(up_size)(x)
    x = layers.Conv2D(
        filters, kernel_size, strides=strides, padding=padding, use_bias=use_bias
    )(x)

    if use_bn:
        x = layers.BatchNormalization()(x)

    if activation:
        x = activation(x)
    if use_dropout:
        x = layers.Dropout(drop_value)(x)
    return x


def get_generator_model():
    noise = layers.Input(shape=(noise_dim,))
    x = layers.Dense(4 * 4 * 256, use_bias=False)(noise)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(0.2)(x)

    x = layers.Reshape((4, 4, 256))(x)
    x = upsample_block(
        x,
        128,
        layers.LeakyReLU(0.2),
        strides=(1, 1),
        use_bias=False,
        use_bn=True,
        padding="same",
        use_dropout=False,
    )
    x = upsample_block(
        x,
        64,
        layers.LeakyReLU(0.2),
        strides=(1, 1),
        use_bias=False,
        use_bn=True,
        padding="same",
        use_dropout=False,
    )
    x = upsample_block(
        x, 1, layers.Activation("tanh"), strides=(1, 1), use_bias=False, use_bn=True
    )
    # At this point, we have an output which has the same shape as the input, (32, 32, 1).
    # We will use a Cropping2D layer to make it (28, 28, 1).
    x = layers.Cropping2D((2, 2))(x)

    g_model = keras.models.Model(noise, x, name="generator")
    return g_model


g_model = define_generator(noise_dim)
g_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 28224)        3640896     input_4[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 128)       2688        input_3[0][0]                    
____________________________________________________________________________________________

In [12]:
class WGAN(keras.Model):
    def __init__(
        self,
        discriminator,
        generator,
        latent_dim,
        discriminator_extra_steps=3,
        gp_weight=10.0,
    ):
        super(WGAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim
        self.d_steps = discriminator_extra_steps
        self.gp_weight = gp_weight

    def compile(self, d_optimizer, g_optimizer, d_loss_fn, g_loss_fn):
        super(WGAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.d_loss_fn = d_loss_fn
        self.g_loss_fn = g_loss_fn

    def gradient_penalty(self, batch_size, real_images, fake_images, real_labels):
        """ Calculates the gradient penalty.

        This loss is calculated on an interpolated image
        and added to the discriminator loss.
        """
        # Get the interpolated image
        alpha = tf.random.normal([batch_size, 1, 1, 1], 0.0, 1.0)
        diff = fake_images - real_images
        interpolated = real_images + alpha * diff

        with tf.GradientTape() as gp_tape:
            gp_tape.watch(interpolated)
            # 1. Get the discriminator output for this interpolated image.
            pred = self.discriminator([interpolated, real_labels], training=True)

        # 2. Calculate the gradients w.r.t to this interpolated image.
        grads = gp_tape.gradient(pred, [interpolated, real_labels])[0]
        # 3. Calculate the norm of the gradients.
        norm = tf.sqrt(tf.reduce_sum(tf.square(grads), axis=[1, 2, 3]))
        gp = tf.reduce_mean((norm - 1.0) ** 2)
        return gp

    def train_step(self, data):
        real_images, real_labels = data

        # Get the batch size
        batch_size = tf.shape(real_images)[0]

        # For each batch, we are going to perform the
        # following steps as laid out in the original paper:
        # 1. Train the generator and get the generator loss
        # 2. Train the discriminator and get the discriminator loss
        # 3. Calculate the gradient penalty
        # 4. Multiply this gradient penalty with a constant weight factor
        # 5. Add the gradient penalty to the discriminator loss
        # 6. Return the generator and discriminator losses as a loss dictionary

        # Train the discriminator first. The original paper recommends training
        # the discriminator for `x` more steps (typically 5) as compared to
        # one step of the generator. Here we will train it for 3 extra steps
        # as compared to 5 to reduce the training time.
        for i in range(self.d_steps):
            # Get the latent vector
            random_latent_vectors = tf.random.normal(
                shape=(batch_size, self.latent_dim)
            )
            with tf.GradientTape() as tape:
                # Generate fake images from the latent vector
                fake_images = self.generator([random_latent_vectors, real_labels], training=True)
                # Get the logits for the fake images
                fake_logits = self.discriminator([fake_images, real_labels], training=True)
                # Get the logits for the real images
                real_logits = self.discriminator([real_images, real_labels], training=True)

                # Calculate the discriminator loss using the fake and real image logits
                d_cost = self.d_loss_fn(real_img=real_logits, fake_img=fake_logits)
                # Calculate the gradient penalty
                gp = self.gradient_penalty(batch_size, real_images, fake_images, real_labels)
                # Add the gradient penalty to the original discriminator loss
                d_loss = d_cost + gp * self.gp_weight

            # Get the gradients w.r.t the discriminator loss
            d_gradient = tape.gradient(d_loss, self.discriminator.trainable_variables)
            # Update the weights of the discriminator using the discriminator optimizer
            self.d_optimizer.apply_gradients(
                zip(d_gradient, self.discriminator.trainable_variables)
            )

        # Train the generator
        # Get the latent vector
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        random_labels = np.random.randint(0, N_CLASSES, batch_size)
        with tf.GradientTape() as tape:
            # Generate fake images using the generator
            generated_images = self.generator([random_latent_vectors, random_labels], training=True)
            # Get the discriminator logits for fake images
            gen_img_logits = self.discriminator([generated_images, random_labels], training=True)
            # Calculate the generator loss
            g_loss = self.g_loss_fn(gen_img_logits)

        # Get the gradients w.r.t the generator loss
        gen_gradient = tape.gradient(g_loss, self.generator.trainable_variables)
        # Update the weights of the generator using the generator optimizer
        self.g_optimizer.apply_gradients(
            zip(gen_gradient, self.generator.trainable_variables)
        )
        return {"d_loss": d_loss, "g_loss": g_loss}

In [13]:
class GANMonitor():
    def __init__(self, model, num_img=100, latent_dim=128):
        self.num_img = num_img
        self.latent_dim = latent_dim
        self.model = model

    def on_epoch_end(self, epoch, logs=None, save_path=''):
        n = int(np.sqrt(self.num_img))
        random_latent_vectors = np.random.normal(size=(self.num_img, self.latent_dim))
        random_labels = np.asarray([x for _ in range(10) for x in range(10)])
        generated_images = self.model.predict([random_latent_vectors, random_labels])
        # scale from [-1,1] to [0,1]
        generated_images = (generated_images + 1) / 2.0
        self._generate_plot(generated_images, n, os.path.join(save_path, f'{epoch}'))
    
    def _generate_plot(self, examples, n, prefix):
        # plot images
        fig = plt.figure(figsize=(12,12))
        for i in range(n * n):
            # define subplot
            plt.subplot(n, n, 1 + i)
            # turn off axis
            plt.axis('off')
            # plot raw pixel data
            plt.imshow(examples[i])
        #pyplot.show()
        fig.savefig(f'{prefix}_image.png')
        plt.close('all')

In [14]:
# Instantiate the optimizer for both networks
# (learning_rate=0.0002, beta_1=0.5 are recommended)
generator_optimizer = keras.optimizers.Adam(
    learning_rate=0.0002, beta_1=0.5, beta_2=0.9
)
discriminator_optimizer = keras.optimizers.Adam(
    learning_rate=0.0002, beta_1=0.5, beta_2=0.9
)

# Define the loss functions for the discriminator,
# which should be (fake_loss - real_loss).
# We will add the gradient penalty later to this loss function.
def discriminator_loss(real_img, fake_img):
    real_loss = tf.reduce_mean(real_img)
    fake_loss = tf.reduce_mean(fake_img)
    return fake_loss - real_loss


# Define the loss functions for the generator.
def generator_loss(fake_img):
    return -tf.reduce_mean(fake_img)


# Set the number of epochs for trainining.
epochs = 20

# Instantiate the customer `GANMonitor` Keras callback.
cbk = GANMonitor(g_model, num_img=100, latent_dim=noise_dim)

# Instantiate the WGAN model.
wgan = WGAN(
    discriminator=d_model,
    generator=g_model,
    latent_dim=noise_dim,
    discriminator_extra_steps=3, # was 3
)

# Compile the WGAN model.
wgan.compile(
    d_optimizer=discriminator_optimizer,
    g_optimizer=generator_optimizer,
    g_loss_fn=generator_loss,
    d_loss_fn=discriminator_loss,
)

# Start training the model.
#wgan.fit(train_images, batch_size=BATCH_SIZE, epochs=epochs, callbacks=[cbk])

In [ ]:
for ep in range(epochs):
    iteration = train_size // BATCH_SIZE
    save_path = os.path.join(SAVE_RESULT, f'ep_{ep}')
    os.makedirs(save_path, exist_ok=True)
    for i in range(iteration):
        real_images, real_labels = list(dataset.take(1))[0]
        data_losses = wgan.train_step([real_images, real_labels])
        print('>%d, %d/%d, d=%.3f, g=%.3f' %
            (ep+1, i+1, iteration, data_losses['d_loss'], data_losses['g_loss']))
        
        if i % 50 == 0:
            cbk.on_epoch_end(f'i_{i}_ep_{ep}', save_path=save_path)

>1, 1/3150, d=7.475, g=0.008
>1, 2/3150, d=-7.111, g=-0.009
>1, 3/3150, d=-115.995, g=0.185
>1, 4/3150, d=-732.639, g=0.802
>1, 5/3150, d=-2739.487, g=-12.291
>1, 6/3150, d=-5390.223, g=-90.120
>1, 7/3150, d=-4984.225, g=-162.336
>1, 8/3150, d=-5972.561, g=-337.095
>1, 9/3150, d=-6110.553, g=-656.532
>1, 10/3150, d=-5265.359, g=-1150.621
>1, 11/3150, d=-5293.562, g=-1574.979
>1, 12/3150, d=-3568.535, g=-2249.475
>1, 13/3150, d=-2908.731, g=-2720.083
>1, 14/3150, d=-1697.838, g=-3012.035
>1, 15/3150, d=-905.138, g=-2650.984
>1, 16/3150, d=-504.702, g=-2496.448
>1, 17/3150, d=-225.867, g=-1980.972
>1, 18/3150, d=-169.837, g=-1521.432


>1, 19/3150, d=-215.268, g=-1231.021
>1, 20/3150, d=-208.585, g=-1005.248
>1, 21/3150, d=-219.454, g=-753.706
>1, 22/3150, d=-293.205, g=-446.545
>1, 23/3150, d=-602.327, g=-115.476
>1, 24/3150, d=-541.052, g=-54.854
>1, 25/3150, d=-424.162, g=-120.391
>1, 26/3150, d=-510.465, g=-50.689
>1, 27/3150, d=-453.716, g=-282.396
>1, 28/3150, d=-505.296, g=-31.458
>1, 29/3150, d=-857.350, g=-213.274
>1, 30/3150, d=-533.047, g=-231.482
>1, 31/3150, d=-603.526, g=-339.962
>1, 32/3150, d=-490.906, g=-257.792
>1, 33/3150, d=-548.124, g=-320.478
>1, 34/3150, d=-1018.627, g=-384.194
>1, 35/3150, d=-2405.309, g=113.499
>1, 36/3150, d=-3136.022, g=218.401
>1, 37/3150, d=-3334.388, g=323.797


>1, 38/3150, d=-3158.974, g=221.814
>1, 39/3150, d=-1817.552, g=717.775
>1, 40/3150, d=-847.314, g=400.353
>1, 41/3150, d=-386.897, g=-98.778
>1, 42/3150, d=-186.248, g=-51.314
>1, 43/3150, d=-118.068, g=37.963
>1, 44/3150, d=-180.655, g=462.918
>1, 45/3150, d=-244.831, g=740.029
>1, 46/3150, d=-285.848, g=981.127
>1, 47/3150, d=-483.956, g=1179.700
>1, 48/3150, d=-422.788, g=1146.330
>1, 49/3150, d=-668.049, g=1531.917
>1, 50/3150, d=-800.277, g=1811.895
>1, 51/3150, d=-924.859, g=793.099
>1, 52/3150, d=-1299.204, g=2143.352
>1, 53/3150, d=-1003.294, g=700.880
>1, 54/3150, d=-915.078, g=2101.395
>1, 55/3150, d=-1289.674, g=1360.593
>1, 56/3150, d=-876.276, g=653.072


>1, 57/3150, d=-1142.156, g=433.132
>1, 58/3150, d=-915.341, g=-1138.336
>1, 59/3150, d=-942.167, g=499.743
>1, 60/3150, d=-909.984, g=-617.278
>1, 61/3150, d=-978.656, g=-453.761
>1, 62/3150, d=-1102.224, g=697.488
>1, 63/3150, d=-1004.971, g=-603.486
>1, 64/3150, d=-1286.945, g=1097.965
>1, 65/3150, d=-1076.482, g=-249.698
>1, 66/3150, d=-1163.412, g=-706.040
>1, 67/3150, d=-1223.844, g=-134.335
>1, 68/3150, d=-1177.736, g=33.121
>1, 69/3150, d=-1040.813, g=4.706
>1, 70/3150, d=-1036.208, g=-195.286
>1, 71/3150, d=-1156.594, g=660.448
>1, 72/3150, d=-1049.891, g=-700.830
>1, 73/3150, d=-1129.562, g=882.462
>1, 74/3150, d=-1034.424, g=198.023
>1, 75/3150, d=-820.041, g=-995.056


>1, 76/3150, d=-1174.870, g=883.832
>1, 77/3150, d=-1039.397, g=-911.055
>1, 78/3150, d=-977.236, g=-610.623
>1, 79/3150, d=-801.441, g=223.250
>1, 80/3150, d=-1265.223, g=810.464
>1, 81/3150, d=-996.370, g=-567.662
>1, 82/3150, d=-1169.560, g=177.598
>1, 83/3150, d=-843.918, g=-666.951
>1, 84/3150, d=-1113.187, g=519.452
>1, 85/3150, d=-785.853, g=-733.393
>1, 86/3150, d=-1239.994, g=20.263
>1, 87/3150, d=-947.134, g=-177.035
>1, 88/3150, d=-1066.383, g=-328.191
>1, 89/3150, d=-961.599, g=113.991
>1, 90/3150, d=-938.340, g=370.165
>1, 91/3150, d=-1188.975, g=389.247
>1, 92/3150, d=-1203.844, g=24.249
>1, 93/3150, d=-915.614, g=-707.001
>1, 94/3150, d=-881.851, g=251.842


...

>2, 1387/3150, d=142.302, g=-726.425
>2, 1388/3150, d=23.528, g=-991.935
>2, 1389/3150, d=-86.868, g=-881.091
>2, 1390/3150, d=12.548, g=-904.631
>2, 1391/3150, d=28.838, g=-569.695
>2, 1392/3150, d=-90.269, g=-615.798
>2, 1393/3150, d=-26.549, g=-859.806
>2, 1394/3150, d=-64.504, g=-1182.913
>2, 1395/3150, d=19.446, g=-1751.795
>2, 1396/3150, d=73.219, g=-1482.711
>2, 1397/3150, d=-45.704, g=-1467.514
>2, 1398/3150, d=-273.233, g=-2892.592
>2, 1399/3150, d=-17.120, g=-1558.645
>2, 1400/3150, d=-693.322, g=-2541.735
>2, 1401/3150, d=-413.108, g=-2663.717
>2, 1402/3150, d=-150.038, g=-4776.549
>2, 1403/3150, d=-175.249, g=-6723.028
>2, 1404/3150, d=-108.355, g=-5130.162


>2, 1405/3150, d=-396.707, g=-3347.303
>2, 1406/3150, d=1975.761, g=-3620.407
>2, 1407/3150, d=-167.367, g=-3306.048
>2, 1408/3150, d=869.246, g=-2553.403
>2, 1409/3150, d=391.554, g=-2057.956
>2, 1410/3150, d=478.781, g=-1447.438
>2, 1411/3150, d=308.625, g=-2643.190
>2, 1412/3150, d=-675.735, g=-3138.303
>2, 1413/3150, d=377.424, g=-3180.704
>2, 1414/3150, d=-223.480, g=-4305.410
>2, 1415/3150, d=-751.178, g=-2618.253
>2, 1416/3150, d=-223.886, g=-2283.445
>2, 1417/3150, d=417.914, g=-2433.831
>2, 1418/3150, d=333.698, g=-1772.238
>2, 1419/3150, d=-14.940, g=-1453.889
>2, 1420/3150, d=8.225, g=-1870.556
>2, 1421/3150, d=9.085, g=-1617.670
>2, 1422/3150, d=-89.846, g=-2177.706
>2, 1423/3150, d=75.789, g=-1670.240


>2, 1424/3150, d=17.034, g=-2150.965
>2, 1425/3150, d=1.297, g=-1560.549
>2, 1426/3150, d=16.927, g=-1015.971
>2, 1427/3150, d=-1.344, g=-1085.179
>2, 1428/3150, d=-19.058, g=-632.527
>2, 1429/3150, d=5.769, g=-664.999
>2, 1430/3150, d=-63.226, g=-777.756
>2, 1431/3150, d=-151.458, g=-1286.724
>2, 1432/3150, d=-60.118, g=-2099.111
>2, 1433/3150, d=159.854, g=-2299.469
>2, 1434/3150, d=3.113, g=-3132.016
>2, 1435/3150, d=-385.732, g=-2667.838
>2, 1436/3150, d=186.162, g=-2200.435
>2, 1437/3150, d=-160.865, g=-2309.184
>2, 1438/3150, d=-551.658, g=-2975.934
>2, 1439/3150, d=-204.505, g=-4972.743
>2, 1440/3150, d=659.199, g=-3803.128
>2, 1441/3150, d=-242.555, g=-4245.751
>2, 1442/3150, d=94.816, g=-4419.634


>2, 1443/3150, d=-193.369, g=-5161.316
>2, 1444/3150, d=-1094.442, g=-5613.758
>2, 1445/3150, d=-752.107, g=-7464.553
>2, 1446/3150, d=670.597, g=-8769.897
>2, 1447/3150, d=7.126, g=-7756.086
>2, 1448/3150, d=-501.347, g=-7726.174
>2, 1449/3150, d=-237.843, g=-6314.466
>2, 1450/3150, d=397.841, g=-8953.509
>2, 1451/3150, d=607.447, g=-6313.576
>2, 1452/3150, d=-43.005, g=-6155.427
>2, 1453/3150, d=-525.026, g=-4512.936
>2, 1454/3150, d=132.245, g=-5769.990
>2, 1455/3150, d=-384.402, g=-5589.693
>2, 1456/3150, d=34.763, g=-5930.736
>2, 1457/3150, d=-928.215, g=-5577.531
>2, 1458/3150, d=-83.626, g=-6392.966
>2, 1459/3150, d=107.691, g=-5065.121
>2, 1460/3150, d=-523.467, g=-4714.403


>2, 1461/3150, d=757.525, g=-5975.542
>2, 1462/3150, d=-402.978, g=-5478.676
>2, 1463/3150, d=-276.085, g=-6886.535
>2, 1464/3150, d=-124.577, g=-5622.834
>2, 1465/3150, d=-1732.192, g=-10743.925
>2, 1466/3150, d=975.862, g=-10635.321
>2, 1467/3150, d=-52.272, g=-8070.115
>2, 1468/3150, d=845.666, g=-9777.906
>2, 1469/3150, d=-161.862, g=-7213.357


In [ ]:
wgan.save_weights('wgan_generator.h5')